# Huffman Coding

The goal of this task is to implement the Huffman Coding algorithm using a priority queue.  
Let's import all the necessary libraries for this task.


In [27]:
import heapq
import math
import collections

The Huffman Coding algorithm requires building the Huffman Tree. Let's create a class representing a single node in that tree.


In [28]:
class Node:
    def __init__(self, char, freq):
        self.char = char  # The character
        self.freq = freq  # The frequency of the character
        self.left = None   # Left child
        self.right = None  # Right child

    # Define comparison operators for heapq to sort nodes by frequency
    def __lt__(self, other):
        return self.freq < other.freq

And now let's create a function for building this tree. The general idea is to always place the two nodes with the lowest frequency at the bottom of the tree and merge them into one node with the summed-up frequency until there is only one node left, which becomes the root.


In [29]:
def create_huffman_code(frequencies):
    # Create a priority queue (min-heap) with Node objects
    priority_queue = [Node(char, freq) for char, freq in frequencies.items()]
    heapq.heapify(priority_queue)
    
    # Build the Huffman tree
    while len(priority_queue) > 1:
        left = heapq.heappop(priority_queue)
        right = heapq.heappop(priority_queue)
        
        merged = Node(None, left.freq + right.freq)
        merged.left = left
        merged.right = right
        
        heapq.heappush(priority_queue, merged)
    
    # Generate the codes from the Huffman tree
    root = priority_queue[0]
    huffman_codes = {}
    
    def generate_codes(node, current_code):
        if node:
            # Generate code only if this is a real character and not a merged node
            if node.char is not None:
                huffman_codes[node.char] = current_code
            generate_codes(node.left, current_code + "0")
            generate_codes(node.right, current_code + "1")
    
    generate_codes(root, "")
    return huffman_codes

And now let's create a function for encoding a given message. For better visualization, I will treat zeros and ones as characters (rather than individual bits). Only later, when calculating the bit size gain from the compression, will I ensure that both the compressed and original texts match their actual sizes.


In [30]:
def encode(text, huffman_codes):
    return ''.join(huffman_codes[char] for char in text)

We also need a function for decoding the encoded text.


In [31]:
def decode(encoded_text, huffman_codes):
    # Reverse the code dictionary to decode
    reverse_codes = {v: k for k, v in huffman_codes.items()}
    
    decoded_text = []
    current_code = ""
    
    for bit in encoded_text:
        current_code += bit
        if current_code in reverse_codes:
            decoded_text.append(reverse_codes[current_code])
            current_code = ""
    
    return ''.join(decoded_text)

And now let's calculate the actual bit size of a message. For the compressed message, it is just the number of 0s and 1s, but for the original message, we need to multiply the length of the message by the fixed-length size of character encoding. Let's assume that our messages will always consist of 26 Latin characters, 10 digits, and 1 space. The closest larger multiple of 2 is 64, which is 2^6, so the fixed-length encoding will be 6 bits long. (It would be 8 bits if we accept all ASCII characters.)


In [32]:
def calculate_compressed_size(encoded_text):
    return len(encoded_text)

def calculate_fixed_size(text, num_bits=6):
    return len(text) * num_bits

Finally, we need a function to display the results of Huffman encoding. The following `display()` function takes in only one parameter, which is the path to the .txt file that contains the text for encoding.


In [33]:
def calculate_fixed_length_bits(num_unique_chars):
    return math.ceil(math.log2(num_unique_chars))

def display(file_path):
    # Read the text from the file
    with open(file_path, 'r') as file:
        text = file.read()

    frequencies = collections.Counter(text)
    huffman_codes = create_huffman_code(frequencies)

    print("Huffman Codes:")
    for char, code in huffman_codes.items():
        print(f"'{char}': {code}")

    encoded_text = encode(text, huffman_codes)
    decoded_text = decode(encoded_text, huffman_codes)
    compressed_size = calculate_compressed_size(encoded_text)
    num_unique_chars = len(frequencies)

    # Calculate the bits needed for fixed-length encoding based on unique character count
    fixed_length_bits = calculate_fixed_length_bits(num_unique_chars)
    fixed_size = calculate_fixed_size(text, num_bits=fixed_length_bits)

    print(f"Compressed Size: {compressed_size} bits")
    print(f"Fixed-Length Encoding Size: {fixed_size} bits")

    # Calculate the ratio between the compressed size and the fixed-size encoding
    if fixed_size != 0:
        compression_ratio = compressed_size / fixed_size
    else:
        compression_ratio = 0

    print(f"Compression Ratio (Compressed / Fixed-Length): {compression_ratio:.2f}")


And finally, let's see the results for the `norm_wiku_sample.txt` file.


In [34]:
display("norm_wiki_sample.txt")

Huffman Codes:
'e': 000
'm': 00100
'y': 001010
'k': 0010110
'4': 001011100
'x': 001011101
'5': 001011110
'3': 001011111
's': 0011
'w': 010000
'b': 010001
'c': 01001
'r': 0101
'o': 0110
'n': 0111
'i': 1000
'd': 10010
'2': 10011000
'9': 10011001
'v': 1001101
'g': 100111
't': 1010
'p': 101100
'f': 101101
'l': 10111
'a': 1100
'h': 11010
'8': 110110000
'j': 110110001
'0': 11011001
'q': 1101101000
'z': 1101101001
'6': 1101101010
'7': 1101101011
'1': 11011011
'u': 110111
' ': 111
Compressed Size: 46489714 bits
Fixed-Length Encoding Size: 64733646 bits
Compression Ratio (Compressed / Fixed-Length): 0.72
